In [1]:
import pandas as pd
import numpy as np

# 导入数据，解析survey_time列的值作为独立的日期列，指定字符集类型
train = pd.read_csv('./data/happiness_train_abbr.csv', parse_dates=['survey_time'], encoding='latin-1')
test = pd.read_csv('./data/happiness_test_abbr.csv', parse_dates=['survey_time'], encoding='latin-1')
train.head()

,id,happiness,survey_type,province,city,county,survey_time,gender,birth,nationality,...,family_income,family_m,family_status,house,car,marital,status_peer,status_3_before,view,inc_ability
0,1,4,1,12,32,59,2015-08-04 14:18:00,1,1959,1,...,60000.0,2,2,1,2,3,3,2,4,3
1,2,4,2,18,52,85,2015-07-21 15:04:00,1,1992,1,...,40000.0,3,4,1,2,1,1,1,4,2
2,3,4,2,29,83,126,2015-07-21 13:24:00,2,1967,1,...,8000.0,3,3,1,2,3,2,1,4,2
3,4,5,2,10,28,51,2015-07-25 17:33:00,2,1943,1,...,12000.0,3,3,1,1,7,2,1,3,2
4,5,4,1,7,18,36,2015-08-10 09:50:00,2,1994,1,...,-2.0,4,3,1,1,1,3,2,3,-8


In [2]:
o=set(train['happiness'])
print(o)

{1, 2, 3, 4, 5, -8}


In [3]:
# 处理异常值
# 将happiness中的‘-8’无法回答，改为‘3’说不上幸福不幸福
train['happiness'] = train['happiness'].replace(-8, 3)

In [4]:
# 检查是否有缺失值
def check_missing(df):
    miss = df.isnull().sum().any()
    if miss:
        total_miss = df.isnull().sum()
        percent = (df.isnull().sum())/(df.isnull().count()*100)
        output = pd.concat([total_miss, percent], axis=1, keys=['Total', 'Percent'])
        data_type = []
        for col in df.columns:
            dType =str(df[col].dtype)
            data_type.append(dType)
        output['Types'] = data_type
        print(output[output['Total'] != 0])
        return np.transpose(output)
    else:
        return False
check_missing(train)

               Total   Percent    Types
work_status     5049  0.006311  float64
work_yr         5049  0.006311  float64
work_type       5049  0.006311  float64
work_manage     5049  0.006311  float64
family_income      1  0.000001  float64


,id,happiness,survey_type,province,city,county,survey_time,gender,birth,nationality,...,family_income,family_m,family_status,house,car,marital,status_peer,status_3_before,view,inc_ability
Total,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Percent,0,0,0,0,0,0,0,0,0,0,...,1.25e-06,0,0,0,0,0,0,0,0,0
Types,int64,int64,int64,int64,int64,int64,datetime64[ns],int64,int64,int64,...,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64


In [5]:
# 填充缺失值，hukou_loc、family_income以均值填充
# train['hukou_loc'] = train['hukou_loc'] .fillna(4)
train['family_income'] = train['family_income'] .fillna(train['family_income'].mean())

In [6]:
# 计算调查者年龄，调查年份-出生年份
import datetime
train['survey_time'] = train['survey_time'].dt.year  # 调查时间datetime转换为年
train['age'] = train['survey_time'] - train['birth']
test['survey_time'] = test['survey_time'].dt.year  
test['age'] = test['survey_time'] - test['birth']
train['age'].head()

0    56
1    23
2    48
3    72
4    21
Name: age, dtype: int64

In [7]:
train.to_csv('./data/train.csv', encoding='utf-8',index=False)